In [1]:
import numpy as np
import subprocess
from PIL import Image
import pyqrcode
from pyzbar.pyzbar import decode
import os

In [2]:
def get_qr_array(m):
    qr = pyqrcode.create(m, error='H')
    t = np.array([[1-int(z) for z in x] for x in qr.text().split('\n')[:-1]])
    return t

In [3]:
def qr_rgb_1_encode(frame, size=(400, 400)):
    m = ''.join(str(x) for x in np.random.randint(0, 2, (10,)).tolist())
    t = get_qr_array(m)
    rgb = np.repeat(t[:, :, np.newaxis], 3, axis=2)
    rgb = np.uint8(rgb * 255)
    img = Image.fromarray(rgb, 'RGB')
    img = img.resize(size)
    frame.paste(img)
    return frame, m

In [4]:
def qr_rgb_3_encode(frame, size=(400, 400)):
    ms = []
    rgb = []
    for i in range(3):
        m = ''.join(str(x) for x in np.random.randint(0, 2, (100,)).tolist())
        ms.append(m)
        rgb.append(get_qr_array(m))
    rgb = np.stack(rgb, axis=2)
    rgb = np.uint8(rgb * 255)
    img = Image.fromarray(rgb, 'RGB')
    img = img.resize(size)
    frame.paste(img)
    return frame, ms

In [5]:
def qr_rgb_1_decode(frame, size=(400, 400)):
    f = frame.crop((0, 0, size[0], size[1]))
    res = decode(f)
    if not res:
        return ''
    return decode(f)[0].data.decode('utf-8')
   

In [6]:
def qr_rgb_3_decode(frame, size=(400, 400)):
    f = np.array(frame.crop((0, 0, size[0], size[1])))
    ms = []
    for i in range(3):
        t = f[:, :, i][:, :, np.newaxis]
        t = np.repeat(t, 3, axis=2)
        t = Image.fromarray(t, 'RGB')
        res = decode(t)
        if res:
            ms.append(res[0].data.decode('utf-8'))
        else:
            ms.append([])
    return ms

In [7]:
a =  [1,3]
a.append([1,2])
print a

[1, 3, [1, 2]]


In [8]:
def put_data(directory, version=0):
    '''embedes all of the png files in a directory with a qr code and saves them in place.
    returns the string representations of the qr codes in a list
    version 0 stands for 1 channel version 1 stands for 2 channel'''
    if version == 0:
        m = len(os.listdir(directory)) * [None]
    if version == 1:
        m = []
    for i, img in enumerate(os.listdir(directory)):
        frame = Image.open(directory + '/' + img)
        if version == 0:
            f, m[i] = qr_rgb_1_encode(frame)
        elif version == 1:
            f, k = qr_rgb_3_encode(frame)
            m.append(k[0])
            m.append(k[1])
            m.append(k[2])
        f.save(directory + '/' + img)
        if not i % 10:
            print i
    return m

In [9]:
def get_data(directory, version=0):
    '''reads the qr codes in all of the png files in the directory
    returns the string representations of the qr codes in a list
    version 0 stands for 1 channel version 1 stands for 2 channel'''
    if version == 0:
        m = len(os.listdir(directory)) * [None]
    if version == 1:
        m = []
    for i, img in enumerate(os.listdir(directory)):
        frame = Image.open(directory + '/' + img)
        if version == 0:
            m[i] = qr_rgb_1_decode(frame)
        elif version == 1:
            k = qr_rgb_3_decode(frame)
            m.append(k[0])
            m.append(k[1])
            m.append(k[2])

    return m
    
    

In [10]:
print 'start'
init_m = put_data('frames',1)
print 'qr codes embedded'
subprocess.call(["ffmpeg", "-i", "frames/image-%04d.png", "-c:v", "libvpx", "temp.webm"])
print "done encoding"
subprocess.call(["ffmpeg", "-i", "temp.webm", "-vf", "scale=800:600", "outframes/image-%04d.png"])
print "done decoding"
final_m = get_data('outframes',1)

ctr = 0
for i, f in zip(init_m, final_m):
    if i == f:
        ctr += 1
print str(ctr * 1.0 /len(init_m) * 100) + "% of the qr codes were decoded"
print ctr

subprocess.call(["rm", "temp.webm"])
subprocess.call(["rm", "outframes/*"])



#frame = Image.open('frames/image-0422.png')
#f, m0 = qr_rgb_3_encode(frame)
#m1 = qr_rgb_3_decode(f)
#print(m0 == m1)

start
0
10
20
30
40
50
60
70
80
90
100
110
120
qr codes embedded
done encoding
done decoding
40.2203856749% of the qr codes were decoded
146


1

In [11]:
frame = Image.open('frames/image-0422.png')
# f = np.array(frame.convert('HSV'))
f = np.array(frame.convert('YCbCr'))
print(np.max(f[:, :, 0]), np.min(f[:, :, 0]))
f[:, :, 0] = np.clip(f[:, :, 0] + 5, 0, 255)
print(np.max(f[:, :, 0]), np.min(f[:, :, 0]))
# img = Image.fromarray(f, 'HSV')
img = Image.fromarray(f, 'YCbCr')
img.convert('RGB')

IOError: [Errno 2] No such file or directory: 'frames/image-0422.png'